In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
basepath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'

In [37]:
# Load the name mapping for the column names
sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
sbe_name_map

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN
5,avgsvCF,"Average Sound Velocity [Chen-Millero, ft/s]",avgsv-C F,"Chen-Millero, ft/s",NaN
6,avgsvDM,"Average Sound Velocity [Delgrosso, m/s]",avgsv-D M,"Delgrosso, m/s",NaN
7,avgsvDF,"Average Sound Velocity [Delgrosso, ft/s]",avgsv-D F,"Delgrosso, ft/s",NaN
8,avgsvWM,"Average Sound Velocity [Wilson, m/s]",avgsv-W M,"Wilson, m/s",NaN
9,avgsvWF,"Average Sound Velocity [Wilson, ft/s]",avgsv-W F,"Wilson, ft/s",NaN


In [4]:
datapath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'

In [5]:
for file in os.listdir(datapath):
    if '.btl' in file:
        print(file)

ar30-03001.btl
ar30-03002b.btl
ar30-03003.btl
ar30-03004.btl
ar30-03005.btl
ar30-03006.btl
ar30-03007.btl
ar30-03008.btl
ar30-03009.btl
ar30-03010.btl
ar30-03011.btl
ar30-03012.btl
ar30-03013.btl
ar30-03014.btl
ar30-03015.btl
ar30-03016.btl
ar30-03017.btl
ar30-03018.btl
ar30-03019.btl
ar30-03020.btl
ar30-03021.btl
ar30-03022.btl
ar30-03023.btl


In [6]:
filename = datapath+'ar30-03001.btl'
filename

'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/ar30-03001.btl'

In [7]:
with open(filename) as file:
    content = file.readlines()

content = [x.strip() for x in content]

In [8]:
header = []
columns = []
data = []
for line in content:
    if line.startswith('*') or line.startswith('#'):
        header.append(line)
    else:
        try:
            float(line[0])
            data.append(line)
        except:
            columns.append(line)

In [9]:
# Parse the data for the start_time
def parse_header(header):
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [13]:
column_dict = {}
for line in columns:
    for i,x in enumerate(line.split()):
        try:
            column_dict[i] = column_dict[i] + ' ' + x
        except:
            column_dict.update({i:x})



In [14]:
hdr = parse_header(header)
hdr

{'Hex name': 'D:\\Data\\ar30-03001.hex',
 'Start Latitude': '62 46.29 N',
 'Start Longitude': '028 39.40 W',
 'Cruise ID': 'ar30-03',
 'Start Time': '2018-06-06T08:16:04Z'}

In [11]:
data

['1    Jun 06 2018    148.209    146.727   62.77390  -28.65319     6.1764     6.1704   3.455169   3.454611    34.9512    34.9511     6.2855    6.88271    19.0368     0.8578 (avg)',
 '09:26:42      0.583      0.577    0.00000    0.00001     0.0097     0.0087   0.000984   0.000855     0.0018     0.0020     0.0063    0.00161     0.1498     0.0315 (sdev)',
 '2    Jun 06 2018    105.773    104.726   62.77408  -28.65296     6.4277     6.4273   3.476623   3.476591    34.9553    34.9554     6.3981    6.84254    19.1611     0.8311 (avg)',
 '09:31:25      0.660      0.653    0.00000    0.00001     0.0005     0.0004   0.000037   0.000046     0.0002     0.0004     0.0038    0.00007     0.0483     0.0100 (sdev)',
 '3    Jun 06 2018     40.563     40.167   62.77418  -28.65280     6.7659     6.7668   3.505544   3.505613    34.9647    34.9645     6.6604    6.78899    21.4223     0.4769 (avg)',
 '09:36:05      1.383      1.369    0.00000    0.00000     0.0028     0.0031   0.000207   0.000233     0.0003

In [15]:
data_dict = {x:[] for x in column_dict.keys()}

for line in data:
    if line.endswith('(avg)'):
        values = list(filter(None,line.split('  ')))
        for i,x in enumerate(values):
            data_dict[i].append(x)
    elif line.endswith('(sdev)'):
        values = list(filter(None,line.split('  ')))
        data_dict[1].append(values[0])
    else:
        pass
            
data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]

In [16]:
results = {}
for key,item in column_dict.items():
    values = data_dict[key]
    results.update({item:values})

In [49]:
df = pd.DataFrame.from_dict(results)

In [50]:
df

,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,Sal00,Sal11,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,34.9512,34.9511,6.2855,6.88271,19.0368,0.8578 (avg)
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,34.9553,34.9554,6.3981,6.84254,19.1611,0.8311 (avg)
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,34.9647,34.9645,6.6604,6.78899,21.4223,0.4769 (avg)
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,34.9734,34.9686,7.0707,6.72998,26.6674,0.1282 (avg)
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,34.9664,34.9663,7.0591,6.69807,28.1697,0.1376 (avg)
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,34.9664,34.9657,7.0726,6.69589,22.2056,0.3952 (avg)
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,34.9665,34.9664,7.0735,6.69567,22.1383,0.3963 (avg)


In [51]:
for key,item in hdr.items():
    df[key] = item

In [52]:
df

,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,...,Sal11,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,...,34.9511,6.2855,6.88271,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,...,34.9554,6.3981,6.84254,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,...,34.9645,6.6604,6.78899,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,...,34.9686,7.0707,6.72998,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,...,34.9663,7.0591,6.69807,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,...,34.9657,7.0726,6.69589,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,...,34.9664,7.0735,6.69567,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z


In [53]:
df.columns.values[2]

'PrDM'

In [55]:
fullname = list(sbe_name_map[sbe_name_map['Short Name'] == 'PrDM']['Full Name'])
fullname

[]

In [57]:
sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == 'prdm') == True]['Full Name']

347    Pressure, Digiquartz [db]
Name: Full Name, dtype: object

In [45]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower())

0              accm
1              accf
2              altm
3              altf
4           avgsvcm
5           avgsvcf
6           avgsvdm
7           avgsvdf
8           avgsvwm
9           avgsvwf
10              bat
11             bat1
12          batdiff
13         wetbattn
14         cstarat0
15         cstarat1
16         cstarat2
17         cstarat3
18         cstarat4
19         cstarat5
20      cstaratdiff
21            xmiss
22           xmiss1
23        xmissdiff
24        wetbtrans
25         cstartr0
26         cstartr1
27         cstartr2
28         cstartr3
29         cstartr4
           ...     
486         turbwet
487    seaowlbbdiff
488           user1
489           user2
490           user3
491           user4
492           user5
493          uexpo0
494          uexpo1
495          uexpo2
496          upoly0
497          upoly1
498          upoly2
499              v0
500              v1
501              v2
502              v3
503              v4
504              v5


In [58]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [59]:
df

,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Salinity, Practical, 2 [PSU]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,...,34.9511,6.2855,6.88271,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,...,34.9554,6.3981,6.84254,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,...,34.9645,6.6604,6.78899,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,...,34.9686,7.0707,6.72998,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,...,34.9663,7.0591,6.69807,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,...,34.9657,7.0726,6.69589,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,...,34.9664,7.0735,6.69567,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z


In [ ]:
df.columns.values.lower()

In [ ]:
column_dict

In [ ]:
data_dict = {}
line1 = data[0].strip()
line1 = line1.split()
line2 = data[1].strip()
line2.split()

for i,x in enumerate(line2.split()):
    line1[i] = line1[i] + ' ' + x
print(line1)

column_dict = {}
for i,x in enumerate(line1):
    column_dict.update({x:i})
    


In [ ]:
column_dict

In [ ]:
lineparts = list(filter(None,line2.split('  ')))
lineparts

In [ ]:
data_dict = {x:[] for x in column_dict.values()}
for line in data:
    if line.endswith('(avg)'):
        for i,x in enumerate(list(filter(None,line.split('  ')))):
            data_dict[i].append(x.strip())
    else:
        data_dict[1] = data_dict[1]+
        print(line.split())
        


In [ ]:
data_dict

In [ ]:
data

In [ ]:
for short_name in sbe_name_map['Short Name']:
    if 
    print(short_name)